<a href="https://colab.research.google.com/github/CathieLC/Wild_notebook_2021-2022/blob/main/2_1_2_ML_Classification_KNN_distances_with_Pokemons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  il est interdit d'utiliser un Pokémon légendaire. Un pokemon est soit légendaire, soit non-légendaire ==> La dernière colonne du dataset indique True ou False suivant les cas.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Appel de mon dataset

df_pkm = pd.read_csv("https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv")
df_pkm.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


### Nettoyage du df

In [ ]:
#recherche les noms de colonnes comportant des valeurs manquantes. Si true = valeurs manquantes
df_pkm.isna().any()

#             False
Name           True
Type 1        False
Type 2         True
HP            False
Attack        False
Defense       False
Sp. Atk       False
Sp. Def       False
Speed         False
Generation    False
Legendary     False
dtype: bool

In [ ]:
df_pkm.shape

(800, 12)

In [ ]:
# Recherche de lignes avec Nan value dans la colonne name : c'est le pokemon 'Primeape' (source : https://www.pokebip.com/pokedex/pokedex_5G_liste_des_pokemon.html)
df_pkm[df_pkm['Name'].isna()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
62,63,NaN,Fighting,NaN,65,105,60,60,70,95,1,False


In [ ]:
#changer la valeur du pokemon sans nom par son nom
df_pkm['Name'][62]='Primeape'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#normalement il n'y a plus de données manquantes dans name
df_pkm[df_pkm['Name'].isna()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary


In [ ]:
# Recherche de lignes avec Nan value dans la colonne Type 2
df_pkm[df_pkm['Type 2'].isna()]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
5,6,Charmeleon,Fire,NaN,58,64,58,80,65,80,1,False
9,10,Squirtle,Water,NaN,44,48,65,50,64,43,1,False
10,11,Wartortle,Water,NaN,59,63,80,65,80,58,1,False
11,12,Blastoise,Water,NaN,79,83,100,85,105,78,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
775,776,Sliggoo,Dragon,NaN,68,75,53,83,113,60,6,False
776,777,Goodra,Dragon,NaN,90,100,70,110,150,80,6,False
788,789,Bergmite,Ice,NaN,55,69,85,32,35,28,6,False
789,790,Avalugg,Ice,NaN,95,117,184,44,46,28,6,False


#### Pour les autres colonnes il n'y a pas de Nan values

In [ ]:
# remplacer les Nan dans type 2 par 'pas de type 2'

df_pkm['Type 2'].fillna('Pas de Type 2', inplace=True)

In [ ]:
df_pkm.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,Pas de Type 2,39,52,43,60,50,65,1,False


## Refaire un dataframe en remplaçant les Pokémons légendaires par les voisins les plus proches

In [ ]:
#Je créé un df de pokemons non légendaires pour ensuite m'en servir comme modèle de ML

pkm_unlegendary = df_pkm[df_pkm['Legendary'] == False]
pkm_unlegendary = pkm_unlegendary.reset_index(drop=True)
pkm_unlegendary

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,Pas de Type 2,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
730,788,Gourgeist Super Size,Ghost,Grass,85,100,122,58,75,54,6,False
731,789,Bergmite,Ice,Pas de Type 2,55,69,85,32,35,28,6,False
732,790,Avalugg,Ice,Pas de Type 2,95,117,184,44,46,28,6,False
733,791,Noibat,Flying,Dragon,40,30,35,45,40,55,6,False


In [ ]:
# je créé mon modèle sur la base de mon df de pokemons non légendaires
model_df_pkm = pkm_unlegendary[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']]



In [ ]:
##j'entraîne mon modèle à chercher les voisins les plus proches sur les non-légendaires
distanceKNN = NearestNeighbors(n_neighbors=3).fit(model_df_pkm)


In [ ]:
# je cherche les pokemons les plus proches en non légendaires et je créé un tuple

pokemons_kneighbors = distanceKNN.kneighbors(df_pkm.loc[df_pkm['Legendary'] == True, ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']])
pokemons_kneighbors

(array([[ 25.96150997,  30.06659276,  32.75667871],
        [ 16.55294536,  21.11871208,  21.11871208],
        [ 11.5758369 ,  27.45906044,  29.54657341],
        [ 42.22558466,  43.08131846,  45.50824101],
        [ 57.12267501,  62.09669879,  66.3777071 ],
        [ 76.24959016,  81.09253973,  88.62279616],
        [ 25.05992817,  25.29822128,  25.3179778 ],
        [ 24.12467616,  28.74021573,  33.9411255 ],
        [ 30.06659276,  30.08321791,  31.19294792],
        [ 46.10856753,  62.49799997,  64.43601477],
        [ 41.08527717,  58.00862005,  58.16356248],
        [ 43.60045871,  44.73253849,  47.95831523],
        [ 64.39720491,  75.00666637,  75.33923281],
        [ 30.43024811,  41.94043395,  43.9886349 ],
        [ 36.19392214,  37.82856064,  41.37632173],
        [ 46.22769733,  47.83304297,  58.94064811],
        [ 25.51470164,  30.5122926 ,  30.88689042],
        [ 35.9722115 ,  48.51803788,  50.55689864],
        [ 36.46916506,  38.13135193,  47.15930449],
        [ 72

In [ ]:
#check type résultat de distanceKNN.kneighbors
type(pokemons_kneighbors)


tuple

In [ ]:
#j'affiche mon df de pokemon legendary = True
pkm_legendary = df_pkm[df_pkm['Legendary'] == True]
pkm_legendary = pkm_legendary.reset_index(drop=True)
pkm_legendary.shape

#j'obtiens ue liste de 66 pokemons legendaires je dois donc avoir un résultat de 66 nouveaux pokemons non légendaires

(65, 12)

In [ ]:
pkm_legendary.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,157,Articuno,Ice,Flying,90,85,100,95,125,85,1,True
1,158,Zapdos,Electric,Flying,90,90,85,125,90,100,1,True
2,159,Moltres,Fire,Flying,90,100,90,125,85,90,1,True
3,163,Mewtwo,Psychic,Pas de Type 2,106,110,90,154,90,130,1,True
4,164,Mega Mewtwo X,Psychic,Fighting,106,190,100,154,100,130,1,True


In [ ]:
# je check les pokemons - le premier proposé en true est :
pkm_legendary.iloc[0, :]


#                  157
Name          Articuno
Type 1             Ice
Type 2          Flying
HP                  90
Attack              85
Defense            100
Sp. Atk             95
Sp. Def            125
Speed               85
Generation           1
Legendary         True
Name: 0, dtype: object

In [ ]:
# il est proposé d'être remplacé par Blastoise

pkm_unlegendary.iloc[264, :]


#                       276
Name          Mega Sceptile
Type 1                Grass
Type 2               Dragon
HP                       70
Attack                  110
Defense                  75
Sp. Atk                 145
Sp. Def                  85
Speed                   145
Generation                3
Legendary             False
Name: 264, dtype: object

In [ ]:
#Maintenant je vais tester avec les pokemons demandés :
# 'Mewtwo','Lugia','Rayquaza','Dialga','Palkia', 'Giratina Origin Forme'

# 1- je créé un df avec ces pk + rest index
Pokemon_concernés = df_pkm[(df_pkm['Name'].isin(['Mewtwo','Lugia','Rayquaza','Dialga','Palkia', 'Giratina Origin Forme']))]

# 2-je cherche les pokemons les plus proches en non légendaires et je créé un tuple (mon modèm est déjà entraîné plus hat sur le df de pkm_unlegendary )
pokemons_kneighbors2 = distanceKNN.kneighbors(Pokemon_concernés.loc[Pokemon_concernés['Legendary'] == True, ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']])

# 3 - je créé un array
my_array = np.array(pokemons_kneighbors2)

# 4 - je ne garde que l'indice index 
my_array[1]

# 5 - Convertion en liste
tup = my_array[1]
array = np.asarray(tup)
ndarray = array.flatten()
my_array_liste = list(ndarray)
print(my_array_liste)


[242.0, 264.0, 23.0, 508.0, 3.0, 260.0, 268.0, 465.0, 7.0, 12.0, 7.0, 653.0, 8.0, 12.0, 653.0, 510.0, 260.0, 159.0]


In [ ]:
# Je créé 2 listes car j'ai pris les deux plus proches voisins des index de pokemons legendaires
# On prends le 1er élément (index 0) et On saute 1, ensuite pareil avec le deuxième élement (indice 1)

Index1 = []
Index2 = []
Index3 = []


for i in range(0,len(my_array_liste),3):
  Index1.append(int(my_array_liste[i]))
print(Index1)

for i in range(1,len(my_array_liste),3):
  Index2.append(int(my_array_liste[i]))
print(Index2)

for i in range(2,len(my_array_liste),3):
  Index3.append(int(my_array_liste[i]))
print(Index3)


[242, 508, 268, 12, 8, 510]
[264, 3, 465, 7, 12, 260]
[23, 260, 7, 653, 653, 159]


In [ ]:
# check longueur des listes pour voir si ça colle biena vec la longueur de mon tableau pkm_legendary
print(len(Pokemon_concernés))
print(len(Index1))
print(len(Index2))
print(len(Index3))

6
6
6
6


In [ ]:
# je mets les index dans un tableau

Index_des_pokemons_en_remp = pd.DataFrame(list(zip(Index1, Index2, Index3)),
               columns =['Index1_le_plus_proche', 'Index2_le_plus_proche', 'Index3_le_plus_proche'])
Index_des_pokemons_en_remp

,Index1_le_plus_proche,Index2_le_plus_proche,Index3_le_plus_proche
0,242,264,23
1,508,3,260
2,268,465,7
3,12,7,653
4,8,12,653
5,510,260,159


In [ ]:
Index_des_pokemons_en_remp.reset_index(inplace=True)
Index_des_pokemons_en_remp

,index,Index1_le_plus_proche,Index2_le_plus_proche,Index3_le_plus_proche
0,0,242,264,23
1,1,508,3,260
2,2,268,465,7
3,3,12,7,653
4,4,8,12,653
5,5,510,260,159


In [ ]:
type(Index_des_pokemons_en_remp)

pandas.core.frame.DataFrame

In [ ]:
#reset index pour avoir une colonne commune pour le merge
Pokemon_concernés.reset_index(inplace=True)


In [ ]:
type(Pokemon_concernés)

pandas.core.frame.DataFrame

In [ ]:
# et enfin on merge les tableaux !!! Index_des_pokemons_en_remp et Pokemon_concernés 
tableau_final=pd.merge(Pokemon_concernés, Index_des_pokemons_en_remp, how='left', on=['index'])
tableau_final

#je ne comprends pas pourquoi j'ai des Nan !!!!!!

,index,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Index1_le_plus_proche,Index2_le_plus_proche,Index3_le_plus_proche
0,162,163,Mewtwo,Psychic,Pas de Type 2,106,110,90,154,90,130,1,True,NaN,NaN,NaN
1,269,270,Lugia,Psychic,Flying,106,90,130,90,154,110,2,True,NaN,NaN,NaN
2,425,426,Rayquaza,Dragon,Flying,105,150,90,150,90,95,3,True,NaN,NaN,NaN
3,540,541,Dialga,Steel,Dragon,100,120,120,150,100,90,4,True,NaN,NaN,NaN
4,541,542,Palkia,Water,Dragon,90,120,100,150,120,100,4,True,NaN,NaN,NaN
5,545,546,Giratina Origin Forme,Ghost,Dragon,150,120,100,120,100,90,4,True,NaN,NaN,NaN


In [ ]:
pkm_unlegendary.reset_index(inplace=True)
pkm_unlegendary.head()

,index,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,4,5,Charmander,Fire,Pas de Type 2,39,52,43,60,50,65,1,False


In [ ]:
pkm_unlegendary.rename(columns={'index':'Index_tableau_de_base'}, inplace=True)
pkm_unlegendary.head()

,Index_tableau_de_base,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,4,5,Charmander,Fire,Pas de Type 2,39,52,43,60,50,65,1,False


In [ ]:
# Je vais afficher deux tabelaux regroupant les premiers pokemons pouvant remplacer les légendaires sélectionnés et un second pour les seconds plus proches voisins

# j'ai besoin de : liste Index1 et Index2 et du df pkm_unlegendary

First_recommandation = pd.DataFrame()
Second_recommandation = pd.DataFrame()
Third_recommandation = pd.DataFrame()

for i in Index1:
   First_recommandation = First_recommandation.append(pkm_unlegendary[pkm_unlegendary['Index_tableau_de_base'] == i], ignore_index=True)

for i in Index2:
   Second_recommandation = Second_recommandation.append(pkm_unlegendary[pkm_unlegendary['Index_tableau_de_base'] == i-1], ignore_index=True)

for i in Index3:
   Third_recommandation = Third_recommandation.append(pkm_unlegendary[pkm_unlegendary['Index_tableau_de_base'] == i-1], ignore_index=True)   
  

In [ ]:
First_recommandation

,Index_tableau_de_base,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,242,249,Mega Houndoom,Dark,Fire,75,90,90,140,90,115,2,False
1,508,547,Cresselia,Psychic,Pas de Type 2,120,70,120,75,130,85,4,False
2,268,280,Mega Blaziken,Fire,Fighting,80,160,80,130,80,100,3,False
3,12,13,Mega Blastoise,Water,Pas de Type 2,79,103,120,135,115,78,1,False
4,8,9,Mega Charizard Y,Fire,Flying,78,104,78,159,115,100,1,False
5,510,549,Manaphy,Water,Pas de Type 2,100,100,100,100,100,100,4,False


In [ ]:
Second_recommandation

,Index_tableau_de_base,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,263,275,Sceptile,Grass,Pas de Type 2,70,85,65,105,85,120,3,False
1,2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
2,464,494,Garchomp,Dragon,Ground,108,130,95,80,85,102,4,False
3,6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
4,11,12,Blastoise,Water,Pas de Type 2,79,83,100,85,105,78,1,False
5,259,269,Mega Tyranitar,Rock,Dark,100,164,150,95,120,71,2,False


In [ ]:
Third_recommandation

,Index_tableau_de_base,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,22,23,Pidgeot,Normal,Flying,83,80,75,70,70,101,1,False
1,259,269,Mega Tyranitar,Rock,Dark,100,164,150,95,120,71,2,False
2,6,7,Charizard,Fire,Flying,78,84,78,109,85,100,1,False
3,652,696,Zweilous,Dark,Dragon,72,85,70,65,70,58,5,False
4,652,696,Zweilous,Dark,Dragon,72,85,70,65,70,58,5,False
5,158,162,Dragonite,Dragon,Flying,91,134,95,100,100,80,1,False
